# PyTorch quickstart

## 1. 导入相关工具

`torch.utils.data.Dataset`：用于存储数据集，包括训练样本以及对应的label

`torch.utils.data.DotaLoader`：用于生成数据集的可迭代对象

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

## 2. 载入数据集

PyTorch提供了许多与AI相关领域的工具库，例如[TorchText](https://pytorch.org/text/stable/index.html), [TorchVision](https://pytorch.org/vision/stable/index.html), and [TorchAudio](https://pytorch.org/audio/stable/index.html), 并包含了一些常用的数据集。

在torchvision中，包含了常用的图像任务数据集，例如CIFAR，COCO等，完整列表：https://pytorch.org/docs/stable/torchvision/datasets.html

在quickstart教程中，使用的是FashionMNIST dataset。每个torchvision的数据集都包含以下两个参数：
- `transform`: 改变训练样本
- `target_transform`: 改变label


In [4]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

`torchvision.datasets`是继承自`torch.utils.data.Dataset`的数据集实现，要实现数据集加载，还需要将`torch.utils.data.Dataset`的数据集实例作为参数传入`torch.utils.data.DataLoader`，`DataLoader`是可用于训练过程的可迭代对象：

In [8]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in train_dataloader:
    print(X.shape)
    print(y.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


## 3. 模型搭建

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using {} device.'.format(device))

# 定义一个多层感知机
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu = nn.Sequential(
            nn.Linear(28*28, 512), 
            nn.ReLU(),
            nn.Linear(512, 512), 
            nn.ReLU(),
            nn.Linear(512, 10), 
            nn.ReLU()
        )
    def forward(self, x):
        x = self.flatten(x)
        output = self.linear_relu(x)
        return output

mlp = MLP().to(device)
print(mlp)

using cuda device.
MLP(
  (flatten): Flatten()
  (linear_relu): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## 4. 定义优化器、损失函数、模型训练、模型测试

在模型的训练过程中，需要循环读取**数据集**，使用**模型**预测训练样本的结果，根据label和**损失函数**计算loss，并使用**优化器**更新模型参数，因此训练函数应传入一下四部分参数：
- dataloader
- model
- loss function
- optimizer

In [20]:
loss_fn= nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(mlp.parameters(), lr = 1e-3)

def train(dataloader, model, loss_fn, optimizer):
    dataset_size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        predict = model(X)
        loss = loss_fn(predict, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")

在测试函数中，需要对测试集中的测试样本预测输出，同时计算精度，平均误差等指标，评估模型性能。

In [26]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test error:\n Accuracy: {100*correct:>0.1f}%, Avg loss:{test_loss:>8f} \n")

In [27]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, mlp, loss_fn, optimizer)
    test(test_dataloader, mlp)
print("Done!")

Epoch 1
-------------------------------
loss: 1.373670  [    0/60000]
loss: 1.505700  [ 6400/60000]
loss: 1.397663  [12800/60000]
loss: 1.602832  [19200/60000]
loss: 1.335567  [25600/60000]
loss: 1.394668  [32000/60000]
loss: 1.277416  [38400/60000]
loss: 1.252012  [44800/60000]
loss: 1.326200  [51200/60000]
loss: 1.435307  [57600/60000]
Test error:
 Accuracy: 55.2%, Avg loss:0.022007 

Epoch 2
-------------------------------
loss: 1.313377  [    0/60000]
loss: 1.446247  [ 6400/60000]
loss: 1.335440  [12800/60000]
loss: 1.553099  [19200/60000]
loss: 1.272557  [25600/60000]
loss: 1.348485  [32000/60000]
loss: 1.227384  [38400/60000]
loss: 1.203458  [44800/60000]
loss: 1.282182  [51200/60000]
loss: 1.398004  [57600/60000]
Test error:
 Accuracy: 56.1%, Avg loss:0.021313 

Epoch 3
-------------------------------
loss: 1.267812  [    0/60000]
loss: 1.401120  [ 6400/60000]
loss: 1.287564  [12800/60000]
loss: 1.512327  [19200/60000]
loss: 1.226459  [25600/60000]
loss: 1.313802  [32000/60000]


## 5. 模型保存

In [50]:
torch.save(mlp.state_dict(), 'checkpoints/00_mlp.pth')

## 6. 模型读取与测试

In [58]:
mlp = MLP()
mlp.load_state_dict(torch.load('checkpoints/00_mlp.pth'))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

mlp.eval()

x, y = test_data[0][0], test_data[0][1]

pred = mlp(x)

pred, y = classes[pred.argmax(1).item()], classes[y]

print('predict class: {}.'.format(pred), 'Label: {}'.format(y))


predict class: Ankle boot. Label: Ankle boot
